In [ ]:
import sys
from pathlib import Path
sys.path.insert(1, Path.cwd().parent.as_posix())

import matplotlib.pyplot as plt
import numpy as np
from utils.oscilloscope import Oscilloscope, plot_data

In [ ]:
files1 = [Oscilloscope.LabDid.load_channel(1, idx, dir="data", del_data=(-1.024 if idx in (0, 14, 15, 16, 17) else None), plot_dir="elab") for idx in range(18)]